In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import itertools as itr

In [2]:
N = 10
J = 1
s = np.ones((N,N), dtype=int)

In [3]:
beta = 0

i = np.random.randint(N)
j = np.random.randint(N)

s[i,j] *= -1

In [3]:
E = 0
for i,j in itr.product(range(N),range(N)):
    E += s[i,j]*(s[(i+1)%N,j] + s[(i-1),j] + s[i,(j+1)%N] + s[i,(j-1)])
print(-J * E)

-400
